In [2]:
import os
import time
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_driver(download_dir):
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.path.abspath(download_dir),
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)

def download_pjm_lmp_csv_selenium(date, save_dir="pjm_data"):
    """
    Download PJM Day-Ahead Hourly LMPs for a specific date (YYYY-MM-DD) using Selenium.
    """
    os.makedirs(save_dir, exist_ok=True)
    driver = setup_driver(save_dir)
    wait = WebDriverWait(driver, 60)

    try:
        driver.get("https://dataminer2.pjm.com/feed/da_hrl_lmps")
        print(f"🌐 Opened PJM site for {date}...")

        # Find the Start Date and End Date inputs
        input_fields = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//input[@type='text']")))
        if len(input_fields) < 2:
            raise Exception("❌ Could not find date fields.")

        # Set start and end dates via JavaScript
        driver.execute_script(f"""
            const inputs = document.querySelectorAll("input[type='text']");
            if (inputs.length >= 2) {{
                inputs[0].value = '{date}';
                inputs[1].value = '{date}';
            }}
        """)

        print("📅 Dates injected.")

        # Click Submit
        submit_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Submit']")))
        submit_btn.click()
        print("📨 Submitted.")

        # Click Export
        export_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Export']")))
        time.sleep(10)
        export_btn.click()
        print("⬇️ Export triggered.")

        # Wait for download to complete and rename the file
        for _ in range(60):
            for fname in os.listdir(save_dir):
                if fname.endswith(".csv") and not fname.endswith(".crdownload"):
                    old_path = os.path.join(save_dir, fname)
                    new_path = os.path.join(save_dir, f"pjm_lmp_{date}.csv")
                    shutil.move(old_path, new_path)
                    print(f"✅ Downloaded and saved as: {new_path}")
                    return new_path
            time.sleep(2)

        print("❌ Download timeout.")
        return None

    except Exception as e:
        print(f"❌ Error downloading {date}: {e}")
        return None
    finally:
        driver.quit()

# Example usage
if __name__ == "__main__":
    download_pjm_lmp_csv_selenium("2024-03-01")


🌐 Opened PJM site for 2024-03-01...
📅 Dates injected.
📨 Submitted.
❌ Error downloading 2024-03-01: 'NoneType' object has no attribute 'is_displayed'
